<a href="https://colab.research.google.com/github/tsm-mehmetakiftasoz/tsm_makif/blob/main/gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Gerekli kütüphaneler
!pip install -q google-genai
import google.generativeai as genai
import ipywidgets as widgets
from IPython.display import display, HTML
from google.colab import output
import time

# 🔑 API ayarı
genai.configure(api_key="xxxx")

# 🌐 Başlangıç bağlantısı
bağlantı_durumu = widgets.HTML("<b style='color:#ff9800;'>🔄 Gemini’ye bağlanılıyor...</b>")

# 🪄 Arayüz
output_area = widgets.Output(layout={
    'border': '3px solid #d97e07',
    'border_radius': '10px',
    'padding': '10px',
    'background_color': '#fff8dc',
    'max_height': '400px',
    'overflow_y': 'auto'
})

metin_kutusu = widgets.Textarea(
    placeholder='💬 Mesajınızı yazın... (Enter = alt satır, Ctrl+Enter = gönder , sohbeti sonlandırmak için "mischief managed." yazın.)',
    layout=widgets.Layout(width='80%', height='80px', padding='5px')
)

gonder_butonu = widgets.Button(
    description='🚀 Gönder',
    button_style='success',
    layout=widgets.Layout(width='20%', height='80px', padding='5px')
)

ui = widgets.VBox([bağlantı_durumu, output_area, widgets.HBox([metin_kutusu, gonder_butonu])])
display(ui)



# 🔗 Gemini bağlantısı
try:
    model = genai.GenerativeModel("gemini-2.5-flash")
    chat = model.start_chat(history=[])
    bağlantı_durumu.value = """
    <b style='color:#ff9800;'>🔄 Gemini’ye bağlanılıyor...</b>
    """

    time.sleep(2)
    bağlantı_durumu.value = """
    <b style='color:#4CAF50;'>✅ Gemini’ye başarıyla bağlanıldı.</b><br>
    <i>I solemnly swear that I am up to no good.</i>
    """
except Exception as e:
    bağlantı_durumu.value = f"<b style='color:red;'>❌ Bağlantı hatası: {e}</b>"

# 🧙‍♂️ Mesaj gösterme
def mesaj_goster(kimden, metin):
    renk = '#30D5C8' if kimden == 'Siz' else '#f5e36c'
    hizalama = 'right' if kimden == 'Siz' else 'left'
    with output_area:
        display(HTML(f"""
        <div class='msg' style='text-align:{hizalama}; margin: 5px; opacity:1; transition:opacity 1.5s;'>
            <div style='display:inline-block; background:{renk}; border-radius:10px; padding:8px 12px; max-width:80%;'>
                <b>{kimden}:</b> {metin.replace('\\n','<br>')}
            </div>
        </div>
        """))


# 🧹 Sohbeti sonlandır
def sohbeti_kapat():
    metin_kutusu.disabled = True
    gonder_butonu.disabled = True
    bağlantı_durumu.value = "<b style='color:#9c27b0;'>✨Mischief managed. Otherwise, evil can see ..  👁️</b>"

# 💨 Fade-out tetikleme
def fade_out_mesajlar():
    display(HTML("""
    <script>
    let msgs = document.querySelectorAll('.msg');
    msgs.forEach(m => m.style.transition = 'opacity 2s');
    setTimeout(() => {
        msgs.forEach(m => m.style.opacity = 0);
    }, 500);
    </script>
    """))

# 🚀 Mesaj gönderme
def mesaj_gonder(_=None):
    kullanici_mesaji = metin_kutusu.value.strip()
    if not kullanici_mesaji:
        return

    mesaj_goster("Siz", kullanici_mesaji)
    metin_kutusu.value = ''

    # Sohbeti sonlandırma
    if kullanici_mesaji.lower() == "mischief managed.":
        fade_out_mesajlar()
        sohbeti_kapat()
        # Placeholder'ı İngilizce alıntıyla güncelle
        metin_kutusu.placeholder = 'Mischief managed. Otherwise, evil can see 👁️'
        time.sleep(3.5)
        output_area.clear_output()
        metin_kutusu.placeholder = ' '
        return

    try:
        yanit = chat.send_message(kullanici_mesaji)
        mesaj_goster("Gemini", yanit.text)
    except Exception as e:
        mesaj_goster("Sistem", f"Hata: {e}")

# 🎯 Gönderme olayları
gonder_butonu.on_click(mesaj_gonder)

# 🎹 Ctrl + Enter = Gönder
def js_klavye_dinleyici():
    display(HTML("""
    <script>
    document.querySelector('textarea').addEventListener('keydown', function(e) {
        if (e.ctrlKey && e.key === 'Enter') {
            google.colab.kernel.invokeFunction('notebook.mesajGonder', [], {});
            e.preventDefault();
        }
    });
    </script>
    """))

output.register_callback('notebook.mesajGonder', mesaj_gonder)
js_klavye_dinleyici()


